# Излечение, преобразование, загрузка

## Импортируем необходимые модули

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import datetime

## Настраиваем подключение к БД

In [2]:
user = "entries_user"
password = "entries_password"
host = "localhost"
port = "5432"
database = "entries_db"

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

## Выгрузка данных из таблицы

In [3]:
df_intervals = pd.read_sql("SELECT * FROM intervals_tgt;", engine, index_col="id")
df_intervals["enter_dt"] = pd.to_datetime(df_intervals["enter_dt"])
df_intervals["exit_dt"] = pd.to_datetime(df_intervals["exit_dt"])

## Разибение интервалов

В качестве аналитического допущения разобьём каждый интервал, содержащий дату со временем 04:00:00 на два интервала, концом первого будет 04:00:00, а началом второго 04:00:01.

Примечание: в нашем случае интервал содержит максимум одну дату со временем 04:00:00, однако задача, подразумевающая большее количество интервалов так же легко решаема при помощи тривиальной итерации от `enter_dt.date()` до `exit_dt.date()` и проверки выполнения того же неравенства
$$ enter\_dt < split\_dt_{i} < exit\_dt, $$
где $split\_dt$ - дата со временем 04:00:00.

In [4]:
to_push = []
to_drop = []

for row in df_intervals.itertuples():
    split_dt = row.exit_dt.replace(hour=4, minute=0, second=0)

    if row.enter_dt < split_dt < row.exit_dt:
        to_drop.append(row.Index)
        to_push.append({
            "full_name" : row.full_name,
            "enter_dt" : row.enter_dt,
            "exit_dt" : split_dt
        })
        to_push.append({
            "full_name" : row.full_name,
            "enter_dt" : split_dt + pd.Timedelta(seconds=1),
            "exit_dt" : row.exit_dt
        })

df_intervals.drop(to_drop, inplace=True)
df_intervals = pd.concat([df_intervals, pd.DataFrame(to_push)], ignore_index=True)

## Формирование рабочего дня

Для точности напомним, что по определению $ enter\_dt < exit\_dt $, тогда введём $ exit\_dt^{shuft} = exit\_dt - shift $, в данном случае $ shift $ соответствует 4 часам и 1 секнуде. Если $ enter\_dt.date() \neq exit\_dt.date() $, то гарантируется, что после группировки по $ exit\_dt - shift $ интервалы будут относить к одному аналитическому дню.
$$ 04:00:01 < enter\_dt < exit\_dt < 04:00:00, $$
$$ 00:00:01 < enter\_dt - shift < exit\_dt - shift < 00:00:00, $$
левая и правая границы неравнества, разумеется, относятся к датам с разниец в один день.

In [5]:
df_intervals["report_date"] = (df_intervals["exit_dt"] - pd.Timedelta(hours=4, seconds=1)).dt.date
to_push = []

for (name, report_date), group in df_intervals.groupby(["full_name", "report_date"]):
    to_push.append({
        "full_name" : name,
        "report_date" : report_date,
        "enter_dt" : group["enter_dt"].min(),
        "exit_dt" : group["exit_dt"].max()
    })

df_workdays = pd.DataFrame(to_push)

### Выгрузка датафрейма в таблицу и .csv-файл

In [6]:
df_workdays = df_workdays.reset_index()
df_workdays.rename(columns={"index" : "id"}, inplace=True)

df_workdays.to_sql("workdays_tgt", engine, if_exists="append", index=False)
df_workdays.to_csv("data/target/workdays_tgt.csv", index=False)